# Quantum Circuit Simulation for Quantum State Discrimination

UQSD for now

## Load the target circuit

In [1]:
nq = 2
ns = 3
seed = 1
case_id = f"q{nq}_n{ns}_s{seed}"
qasm_name = f"qc_iso_{case_id}_no_backend.qasm"

In [2]:
from qiskit.circuit import QuantumCircuit

qc = QuantumCircuit.from_qasm_file(qasm_name)
print(qc.count_ops())
print(qc.depth())

OrderedDict({'u': 70, 'cx': 47, 'rz': 7})
96


In [3]:
import logging

logging.basicConfig(
    filename=f"sim_{case_id}.log",
    filemode="a",
    format="{asctime} {levelname} {filename}:{lineno}: {message}",
    datefmt="%Y-%m-%d %H:%M:%S",
    style="{",
    level=logging.INFO,
    encoding="utf-8",
)
logger = logging.getLogger(__name__)

## Warm up: some quick checks

In [4]:
# Debug POVM and Phi_tilde
import numpy as np

np.set_printoptions(precision=4)
Phi = np.load(f"Phi_{case_id}.npy")
Phi_tilde = np.load(f"Phi_tilde_{case_id}.npy")
povm = np.load(f"povm_{case_id}.npy")

print(Phi)  # target states
print(Phi_tilde)
print(povm)
np.matmul(Phi_tilde.conj(), Phi)  # Should be identity

[[ 0.7006-0.0719j  0.3096+0.0719j -0.2689-0.6877j]
 [-0.5377+0.2181j -0.2839-0.3145j -0.1648+0.0849j]
 [ 0.3139+0.0322j -0.1023-0.6693j -0.261 -0.4361j]
 [ 0.161 -0.2046j  0.42  -0.2911j  0.3049-0.263j ]]
[[-0.1203+0.2076j -0.8167+0.7522j  0.8676+0.4082j  0.4677-0.6611j]
 [ 0.3483+0.7509j  0.3331-0.0642j -0.1808-1.187j  -0.2086-0.643j ]
 [-0.8597-0.9186j -0.7606-0.2354j  0.0821+0.5225j  1.0824+0.1863j]]
[[-0.0535-0.0924j -0.3635-0.3348j  0.3862-0.1817j  0.2082+0.2942j]
 [ 0.1692-0.3649j  0.1619+0.0312j -0.0879+0.5768j -0.1013+0.3125j]]


array([[ 1.0000e+00-5.8287e-16j,  5.5511e-17-4.9960e-16j,
        -8.0491e-16-7.7716e-16j],
       [-2.8103e-16+2.0817e-16j,  1.0000e+00-4.1633e-17j,
         4.9960e-16+1.5266e-16j],
       [-2.8536e-16+1.6653e-16j,  0.0000e+00-1.1102e-16j,
         1.0000e+00-1.9429e-16j]])

In [5]:
import sys
import time
import tracemalloc
import qiskit.transpiler
import qiskit.synthesis
import qiskit
from qiskit import transpile

qc = transpile(qc, basis_gates=["rx", "ry", "rz", "cx"])
print(int(qc.depth() * 0.05))
qc_approx = transpile(
    circuits=qc,
    unitary_synthesis_method="aqc",
    unitary_synthesis_plugin_config={
        "network_layout": "cart",
        "connectivity_type": "star",
        "depth": int(qc.depth() * 0.05),
    },
)
print("Approx")
print(qc_approx.count_ops())
print(qc_approx.depth())

8
Approx
OrderedDict({'u3': 53, 'cx': 47, 'rz': 7, 'ry': 2, 'rx': 2})
93


In [6]:
## # [:-15] Remove "no_backend.qasm" and add "ibm_brisbane.qasm"
qc_full = QuantumCircuit.from_qasm_file(qasm_name[:-15] + "ibm_brisbane.qasm")
print(qc_full.count_ops())
print(qc_full.depth())

OrderedDict({'rz': 344, 'sx': 216, 'ecr': 101, 'x': 23})
448


## Define handy simulation backends

In [7]:
from qiskit_aer import AerSimulator

AerSimulator().available_methods()

('automatic',
 'statevector',
 'density_matrix',
 'stabilizer',
 'matrix_product_state',
 'extended_stabilizer',
 'unitary',
 'superop')

In [8]:
sv_backend = AerSimulator(method="statevector", seed_simulator=42)
dm_backend = AerSimulator(method="density_matrix", seed_simulator=42)
sv_gpu_backend = AerSimulator(method="statevector", device="GPU", seed_simulator=42)
dm_gpu_backend = AerSimulator(method="density_matrix", device="GPU", seed_simulator=42)

## Load states

In [9]:
from problem_spec import *

In [10]:
states = ProblemSpec.gen_states(
    num_qubits=nq,
    num_states=ns,
    seeds=get_random_seeds(ns, seed=seed),
    state_type="statevector",
)

# Ideal sim verification

Our original circuit does not contain final measurements

It is copied because measure_active creates new ClassicalRegisters every time

https://docs.quantum.ibm.com/guides/measure-qubits

The state discrimination circuit without the initial state

In [11]:
qc_tmp = qc_approx.copy()
qc_tmp.save_statevector()
qc_tmp.measure_active()

# Assume the job always got done quickly
result = sv_backend.run(qc_tmp).result()

In [12]:
result.get_counts()

{'010': 1, '000': 4, '100': 192, '001': 167, '011': 660}

In [13]:
result.get_statevector()

Statevector([-9.3077e-02-5.2359e-02j, -3.9749e-02-4.0023e-01j,
              8.2738e-03-4.8842e-03j,  6.7787e-01-4.0016e-01j,
              3.9189e-01-2.3135e-01j,  9.7145e-17+1.6653e-16j,
              1.6447e-16+2.2786e-16j, -4.5262e-17+6.2578e-17j],
            dims=(2, 2, 2))


## Construct DUT
The circuit with one of the target initial states

In [14]:
from qiskit_aer.library import SetStatevector, set_statevector, SetDensityMatrix
from qiskit.quantum_info import Statevector

dut = QuantumCircuit(qc.num_qubits)
inst = dut.set_statevector(states[0].expand(Statevector([1, 0]))).instructions
dut.append(inst[0], [_ for _ in range(qc.num_qubits)])
# dut.save_statevector()
dut.append(qc, [_ for _ in range(qc.num_qubits)])
dut.save_statevector()
dut.measure_active()

In [15]:
dut.decompose(reps=1).draw(style="mpl")  # May be large
dut.draw(style="mpl")

░  ░ ┌──────────────┐ statevector  ░ ┌─┐      
      q_0: ─░──░─┤0             ├──────░───────░─┤M├──────
            ░  ░ │              │      ░       ░ └╥┘┌─┐   
      q_1: ─░──░─┤1 circuit-160 ├──────░───────░──╫─┤M├───
            ░  ░ │              │      ░       ░  ║ └╥┘┌─┐
      q_2: ─░──░─┤2             ├──────░───────░──╫──╫─┤M├
            ░  ░ └──────────────┘      ░       ░  ║  ║ └╥┘
measure: 3/═══════════════════════════════════════╩══╩══╩═
                                                  0  1  2

In [16]:
# TODO Sim DUT
result = sv_backend.run(dut.decompose(reps=3)).result()

In [17]:
result.get_counts()

{'011': 358, '000': 203, '010': 52, '100': 411}

In [18]:
result.get_statevector()

Statevector([ 3.6122e-01+2.6005e-01j, -9.1113e-09+8.1078e-09j,
              7.7778e-02-2.0079e-01j,  5.5581e-01+1.6947e-01j,
              4.7390e-01+4.3966e-01j, -1.0288e-16+2.8879e-16j,
             -3.7919e-16-1.1969e-17j,  4.6713e-16-6.0059e-16j],
            dims=(2, 2, 2))


In [19]:
# TODO
custom_backend = AerSimulator(
    configuration=None,
    properties=None,
    provider=None,
    method="density_matrix",
    noise_model="",
)

# Noisy sim extrapolation

In [20]:
from qiskit.providers import fake_provider

fake_backend = qiskit.providers.fake_provider.Fake127QPulseV1()

In [21]:
# noise_model =  NoiseModel.from_backend()